1.Understanding of Network Programming and Socket
=======
---
### 1. OSI 7 Layers
### 2. Socket API
### 3. Classfication of Socket Programming
### 4. Procedure of Socket communication
### 5. Example Program : Setting Server, Client
### 6. Host byte sequence <arpa/inet.h>
---
#### 1. OSI 7 Layers

 - 네트워크에서 통신이 일어나는 과정을 7단계로 나눈 것
 - 레퍼런스 모델
 
 ![enter image description here](https://www.webopedia.com/imagesvr_ce/4354/7-layers-of-osi-icon.jpg)

 1. Physical Layer
	 - 역활 : 전기적, 기계적 특성을 이용해서 **통신 케이블**로 데이터 전송(비트 단위)하는 기능(어떤 데이터인지는 상관 없음.)
	 - 장비 : 통신 케이블, 리피터, 허브
 2. Data Link Layer
	 - 역활 : Physical Layer를 통해 송수신되는 정보의 오류 검출 및 흐름관리. 안전한 정보전달을 수행할 수 있도록 도움을 주며 통신 오류 검출 및 재전송을 함. **MAC 주소**를 이용하여 통신
	 - 장비 : 브리지, 스위치
 3.  Network Layer
	 - 역활 : 데이터를 목적지까지 전달하는 기능. 경로 선택 후 경로에 따라 패킷을 전송. **IP 주소**를 사용하고 다양한 프로토콜 및 라우팅 기술 사용.
	 - 장비 : 라우터
 4.  etc
	 - Transport Layer : 최종 시스템 및 호스트 간의 데이터 전송 조율을 담당. **TCP/UDP 프로토콜** 사용.
	 - Session Layer : 통신 세션을 구성하는 계층으로 **Port**연결 후 사용자간의 포트 연결이 유효한지 확인하고 설정.
	 - Presentation Layer : 운영체제의 일부분이며 **네트워크 형식을 응용 프로그램 형식이랑 쌍방 변환**
	 - Application Layer : 응용 프로세스간의 정보 교환 및 각종 응용 프로그램 담당.

#### 2. Socket API
socket이란 네트워크(인터넷)의 연결 도구. 운영체제에 의해 제공이 되는 소프트웨어적인 장치. 소켓은 프로그래머가 데이터 송수신에 대해 물리적, 소프트웨어적인 세부내용을 신경 쓰지 않게 한다. (API 존재)
네트워크 상에서 호스트간의 통신을 가능하게 해주는 일반적인 인터페이스. Between Application Layer with Transport Layer.

#### 3. Classfication of Socket Programming

 1. 네트워킹을 위한 응용프로그램 개발 : http, smtp, ftp, and so on.
 2. Client-Server communication model
 3. User-level Programming

#### 4. Procedure of Socket communication
![enter image description here](http://cfile225.uf.daum.net/image/131A30124C731D49A63FAB)

 - Example of Server Program
        1) Creating socket
```c
#include <stdio.h>
int socket(int domain, int type, int protocol);
```
	 - domain(sys/types.h) : 
         AF_INET(IPv4)
         AF_INET6(IPv6)
         AF_UNIX(프로토콜 내부)
	 - type :
         SOCK_STREAM(TCP)
         SOCK_DGRAM(UDP)
         SOCK_RAW(사용자 정의)
	 - protocol :
         0을 쓰면 시스템이 알아서 세팅
        
        2) binding reference of socket
```c
struct sockaddr {
	u_short sa_family; /* address family (2 bytes) */
	char sa_data[14]; /* IP address + Port Number (14 bytes) */
}
```
	 - 소켓의 주소를 담는 기본 구조체 (16 bytes)
	 - sa_family 변수가 AF_INET인 경우에 는 sockaddr_in 구조체 (16 bytes)사용
	 - sockaddr을 사용할 경우, sa_data에 IP주소와 Port 번호가 조합되어 있어 불편함.
	 - IPv4 주소 체계를 사용하기 쉽게 하기 위해 sockaddr_in 구조체를 사용함.
```c
struct sockaddr_in {
	short sin_family; /* 2 bytes */
	char sa_data[14]; /* IP address + Port Number (14 bytes) */
	struct in_addr sin_addr; /* 4 bytes */
	char sin_zero[8]; /* 8 bytes  (dump)*/
}
struct in_addr {
	u_long s_addr; /* 4 bytes */
}
```
	 - sin_zero 멤버는 0으로 초기화 해야함.
	 3) setting socket address
```d
#include <sys/socket.h>
int bind(int sockfd, struct sockaddr *myaddr, socklen_t addrlen);
```
	 - 성공 시 0, 실패 시 -1 반환 
	 - sockfd :
         소켓의 식별자
	 - *myaddr :
	 - sockaddr_in 구조체 참조(서버용)
	 - addlen : 
          sizeof(struct sockaddr)
	 4) listening connection signals
```d
#include <sys/socket.h>
int listen(int sockfd, int backlog);
```
	 - 성공시 0, 실패 시 -1 반환
	 - sockfd : 연결요청이 가능한 상태의 소켓.
	 - backlog : 연결 요청 대기 큐의 수
	 - 소켓에 할당된 IP와 PORT번호로 연결요청이 가능한 상태가 된다.
	 5) accepting the connection
```c
#include <sys/socket.h>
int accept(int sockfd, struct sockaddr *addr, socklen_t *addrlen)
```
	 - 클라이언트의 connect()요청에 대해서 수락한다.
	 - 연결요청이 수락되면 데이터의 양방향 송수신이 가능하다.
	 6) connecting with server
```c
#include <sys/socket.h>
int connect(int sockfd, struct sockaddr *serv_addr, socklen_t addrlen);
```
	 - 성공 시 0, 실패 시 -1 반환


 - Summary
	 1. 소켓생성 socket()
	 2. IP주소와 Port번호 할당 bind()
	 3. 연결요청 가능상태로 변경 listen()
	 4. 연결요청 수락 accept()

	- 서버는 클라이언트보다 먼저 실행된다.
	- 이렇게 생성된 소켓을 가리켜 서버 소켓 또는 리스닝 소켓이라 한다.

#### 5. Example Program : Setting Server, Client
< hello_server.c >
https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch1/hello_server.c
< hello_client.c >
https://github.com/ndo04343/Learning_Basic-Network-Programming/blob/master/src/ch1/hello_client.c

#### 6. Host byte sequence <arpa/inet.h>
- Big-Endian : IBM, ARM, Motorola (네트워크 바이트)
- Little-Endian : intel x86, AMD, EDC (호스트 바이트)
	
htons() : host to network (short type)
htonl() : host to network (long type)
...
(호스트 바이트 순서를 네트워크 바이트 순서로 변환)
ntohs() : network to host (short type)
ntohl() : network to host (long type)
...
(네트워크 바이트 순서를 호스트 바이트 순서로 변환)

